In [159]:
import json
from pathlib import Path

import numpy as np
import pandas as pd

In [160]:
SEASON = "2025-2026"
SEASON_LABEL = "2025/26"
REPO_ROOT = Path.cwd().resolve()
while REPO_ROOT != REPO_ROOT.parent and not (REPO_ROOT / "config").exists():
    REPO_ROOT = REPO_ROOT.parent
SEASON_DIR = REPO_ROOT / SEASON
DIVISIONS_CONFIG_PATH = REPO_ROOT / "config" / "divisions" / f"{SEASON}.json"

with DIVISIONS_CONFIG_PATH.open(encoding="utf-8") as fp:
    divisions_config = json.load(fp)

all_divisions = {
    division["name"]: division["id"]
    for division in divisions_config.get("divisions", [])
    if division.get("enabled", True)
}

In [161]:
summary_directory = SEASON_DIR / "summary_df"
schedules_directory = SEASON_DIR / "schedules_df"

In [162]:
awaiting_results_directory = SEASON_DIR / "awaiting_results"

In [163]:
schedules_dfs = []
for division in all_divisions.keys():
    for week in range(25, 0, -1):
        file_path = schedules_directory / f"week_{week}" / f"{division}_schedules_df.csv"
        if file_path.exists():
            schedules_df = pd.read_csv(file_path)
            schedules_df["Division"] = division
            schedules_dfs.append(schedules_df)
            break

schedules_df = pd.concat(schedules_dfs, ignore_index=True)
schedules_df = schedules_df.sort_values(by=["Division", "Match Week", "Date"])

schedules_output_path = SEASON_DIR / f"updated_schedule_{SEASON}.csv"
schedules_df.to_csv(schedules_output_path, index=False)

schedules_df["Date"] = pd.to_datetime(schedules_df["Date"], dayfirst=True)

In [164]:
summary_dfs = []
for division in all_divisions.keys():
    for week in range(25, 0, -1):
        file_path = summary_directory / f"week_{week}" / f"{division}_summary_df.csv"
        if file_path.exists():
            summary_df = pd.read_csv(file_path)
            summary_df["Division"] = division
            summary_df["Rank"] = summary_df["Points"].rank(ascending=False, method="min").astype(int)
            summary_df["Teams"] = len(summary_df)
            summary_dfs.append(summary_df)
            break

summary_df = pd.concat(summary_dfs, ignore_index=True)

In [165]:
awaiting_results_dfs = []
for division in all_divisions.keys():
    for week in range(25, 0, -1):
        file_path = awaiting_results_directory / f"week_{week}" / f"{division}_awaiting_results.csv"
        if file_path.exists():
            awaiting_results_df = pd.read_csv(file_path)
            awaiting_results_df["Division"] = division
            awaiting_results_dfs.append(awaiting_results_df)
            break

awaiting_results_df = pd.concat(awaiting_results_dfs, ignore_index=True)

### Create dataframe just for HKCC teams

In [166]:
hkcc = "Hong Kong Cricket Club"

# Filter rows where Team column contains "Hong Kong Cricket Club"
hkcc_summary_df = summary_df[(summary_df["Team"].str.contains(hkcc)) |
                             (summary_df["Team"].str.contains("hkcc", case=False))].reset_index()

hkcc_df = hkcc_summary_df[["Division", "Team", "Played", "Won", "Lost", "Points", "Rank", "Teams"]].copy()

division_order = [
    "Premier Main",
    "2",
    "4",
    "6",
    "7",
    "10",
    "15B",
    "Premier Masters",
    "M2",
    "M3",
    "M4",
    "Premier Ladies",
    "L2",
    "L3",
    "L4",
]

hkcc_df["Division"] = pd.Categorical(hkcc_df["Division"], categories=division_order, ordered=True)
hkcc_df = hkcc_df.sort_values(by=["Division", "Team", "Rank"], ignore_index=True)

hkcc_df["Team"] = hkcc_df["Team"].str.replace("Hong Kong Cricket Club", "HKCC", regex=False)

hkcc_df

,Division,Team,Played,Won,Lost,Points,Rank,Teams
0,Premier Main,HKCC D1,1,1,0,3,1,6
1,2,HKCC D2,2,0,2,2,4,5
2,4,HKCC D4A,2,0,2,2,10,10
3,4,HKCC D4B,2,1,1,7,5,10
4,6,HKCC D6,3,1,2,7,8,12
5,7,HKCC D7A,3,2,1,10,2,11
6,7,HKCC D7B,3,1,2,7,8,11
7,10,HKCC D10,2,1,1,8,6,11
8,15B,HKCC D15,4,4,0,20,1,8
9,Premier Masters,HKCC M1A,3,2,1,9,2,5


In [167]:
def highlight_unbeaten(row):
    if row["Lost"] == 0:
        return ["background-color: #FFFFC5"] * len(row)
    return [""] * len(row)

report_month = pd.Timestamp.now().strftime("%Y_%m")
outputs_dir = REPO_ROOT / "outputs"
outputs_dir.mkdir(parents=True, exist_ok=True)
output_filename = f"hkcc_overall_standings_{report_month}.html"
output_path = outputs_dir / output_filename

styled_html = (
    hkcc_df.style
    .apply(highlight_unbeaten, axis=1)
    .set_table_styles([
        {"selector": "th", "props": [("background-color", "#003366"), ("color", "white"), ("padding", "10px"), ("font-size", "14px")]},
        {"selector": "td", "props": [("padding", "8px"), ("font-size", "13px")]},
        {"selector": "tbody tr:nth-child(even)", "props": [("background-color", "#e6f2ff")]}
    ])
    .set_properties(**{
        "text-align": "center",
        "border": "1px solid #ccc"
    })
    .hide(axis="index")
    .to_html()
)

html_output = f"""
<html>
<head>
    <meta charset=\"UTF-8\">
    <title>HK Squash League {SEASON_LABEL} Standings</title>
    <style>
        body {{
            font-family: Arial, sans-serif;
            padding: 20px;
        }}
        .wrapper {{
            max-width: 800px;
            margin: 0 auto;
        }}
        .header {{
            display: flex;
            align-items: center;
            gap: 20px;
            margin-bottom: 20px;
        }}
        .header img {{
            height: 70px;
        }}
        .title-block {{
            text-align: left;
        }}
    </style>
</head>
<body>

    <div class=\"wrapper\">
        <div class=\"header\">
            <img src=\"hkcc_logo.png\" alt=\"HKCC Logo\">
            <div class=\"title-block\">
                <h2 style=\"color: #003366; margin: 0;\">HK Squash League {SEASON_LABEL} Standings</h2>
            </div>
        </div>

        {styled_html}
    </div>

</body>
</html>
"""

with output_path.open("w", encoding="utf-8") as f:
    f.write(html_output)


In [168]:
# Count the number of rows where Rank is 4 or less
len(hkcc_summary_df[hkcc_summary_df["Rank"] <= 4])

11

In [169]:
kcc = "Kowloon Cricket Club"

# Filter rows where Team column contains "Hong Kong Cricket Club"
kcc_summary_df = summary_df[summary_df["Team"].str.contains(kcc)].reset_index()

kcc_summary_df["Win %"] = kcc_summary_df["Won"] / kcc_summary_df["Played"] * 100

kcc_summary_df[["Division", "Team", "Played", "Won", "Lost", "Points", "Rank", "Teams", "Win %"]]

,Division,Team,Played,Won,Lost,Points,Rank,Teams,Win %
0,2,Kowloon Cricket Club 2,3,3,0,16,1,5,100.000000
1,6,Kowloon Cricket Club 6B,3,3,0,16,1,12,100.000000
2,6,Kowloon Cricket Club 6A,2,1,1,7,8,12,50.000000
3,3,Kowloon Cricket Club 3B,3,2,1,9,3,8,66.666667
4,3,Kowloon Cricket Club 3A,2,1,1,7,6,8,50.000000
5,4,Kowloon Cricket Club 4,3,3,0,15,1,10,100.000000
6,L2,Kowloon Cricket Club L2B,2,2,0,7,2,6,100.000000
7,L2,Kowloon Cricket Club L2A,2,1,1,6,3,6,50.000000
8,9,Kowloon Cricket Club 9,3,1,2,7,8,10,33.333333
9,Premier Main,Kowloon Cricket Club 1A,0,0,0,0,3,6,NaN


### Filter schedules_df for HKCC teams

In [170]:
# Filter schedules_df for rows where Home Team column or Away Team column contains "Hong Kong Cricket Club" or "HKCC"
hkcc_schedules_df = schedules_df[(schedules_df["Home Team"].str.contains(hkcc)) |
                                    (schedules_df["Home Team"].str.contains("hkcc", case=False)) |
                                    (schedules_df["Away Team"].str.contains(hkcc)) |
                                    (schedules_df["Away Team"].str.contains("hkcc", case=False))]

### Create results_df from hkcc_schedules_df

In [171]:
def parse_result(result):
    """
    Function to parse the 'result' string
    """
    overall, rubbers = result.split('(')
    rubbers = rubbers.strip(')').split(',')
    return overall, rubbers

In [172]:
def count_games_won(row):
    """
    Function to count the number of games won by each team in a match,
    handling walkovers (WO) and conceded rubbers (CR) by referring to the 'Overall Score'.
    """
    home_games_won = 0
    away_games_won = 0

    # Calculate the games won from the rubbers, excluding 'CR' and 'WO'
    for rubber in row['Rubbers']:
        if rubber == 'CR' or rubber == 'WO':
            continue
        home, away = map(int, rubber.split('-'))
        home_games_won += home
        away_games_won += away

    # Now handle the 'WO' and 'CR' rubbers by referring to the 'Overall Score'
    if 'WO' in row['Rubbers'] or 'CR' in row['Rubbers']:
        home_overall_score, away_overall_score = map(int, row['Overall Score'].split('-'))
        
        # If the home team has a higher overall score, award the missing games to them
        # Otherwise, award the missing games to the away team
        for rubber in row['Rubbers']:
            if rubber == 'WO' or rubber == 'CR':
                if home_overall_score > away_overall_score:
                    home_games_won += 3
                else:
                    away_games_won += 3

    return home_games_won, away_games_won

In [173]:
# Exclude rows where 'Away Team' is '[BYE]' (indicative of a bye week)
results_df = hkcc_schedules_df[hkcc_schedules_df['Away Team'] != '[BYE]'].copy()

# Replace NaN values in 'Result' with an empty string before applying str.contains
results_df['Result'] = results_df['Result'].fillna('')

# Keep rows where 'Result' contains brackets (indicative of a played match)
results_df = results_df[results_df['Result'].str.contains(r'\(')]

# Apply the function to the 'Result' column
results_df[['Overall Score', 'Rubbers']] = results_df['Result'].apply(lambda x: pd.Series(parse_result(x)))

# Drop the original 'Result' column
results_df.drop(columns=['Result'], inplace=True)

# Replace 'CR' with NaN
results_df.replace('CR', np.nan, inplace=True)
results_df.replace('WO', np.nan, inplace=True)

# Count the number of Rubbers For and Against for each team

# Splitting the 'Overall Score' into two separate columns
results_df[['Home Score', 'Away Score']] = results_df['Overall Score'].str.split('-', expand=True).astype(int)

# Initialize dictionaries to keep track of won and conceded rubbers
rubbers_won = {}
rubbers_conceded = {}

# Create Games Won columns
results_df[['Home Games Won', 'Away Games Won']] = results_df.apply(count_games_won, axis=1, result_type='expand')

In [174]:
# Create Home Win column, using Home Games Won vs Away Games Won as tiebreaker
for i, row in results_df.iterrows():
    if row['Home Score'] > row['Away Score']:
        results_df.loc[i, 'Home Win'] = 1
    elif row['Home Score'] < row['Away Score']:
        results_df.loc[i, 'Home Win'] = 0
    else:
        if row["Home Games Won"] > row["Away Games Won"]:
            results_df.loc[i, 'Home Win'] = 1
        elif row["Home Games Won"] < row["Away Games Won"]:
            results_df.loc[i, 'Home Win'] = 0
        else:
            results_df.loc[i, 'Home Win'] = "Don't know"


In [175]:
# Find the row where Home Win is "Don't know"
results_df[results_df["Home Win"] == "Don't know"]

,Home Team,vs,Away Team,Venue,Time,Match Week,Date,Division,Overall Score,Rubbers,Home Score,Away Score,Home Games Won,Away Games Won,Home Win


In [176]:
# For the row where Home Win is "Don't know", change Home Win to 0, Home Points to 2, and Away Points to 3
results_df.loc[results_df["Home Win"] == "Don't know", "Home Win"] = 0
results_df.loc[results_df["Home Win"] == "Don't know", "Home Points"] = 2
results_df.loc[results_df["Home Win"] == "Don't know", "Away Points"] = 3

In [177]:
results_df["Home Win"] = results_df["Home Win"].astype(int) # if this leads to error, we have an unknown 'tied' result to investigate

In [178]:
# Function to create Home Points and Away Points columns
def create_points_columns(df):

    df['Home Points'] = 0
    df['Away Points'] = 0

    for i, row in df.iterrows():
        if row['Home Score'] > row['Away Score']:
            df.at[i, 'Home Points'] = row["Home Score"] + 1
            df.at[i, 'Away Points'] = row["Away Score"]
        elif row['Home Score'] < row['Away Score']:
            df.at[i, 'Home Points'] = row["Home Score"]
            df.at[i, 'Away Points'] = row["Away Score"] + 1
        else:
            if row["Home Games Won"] > row["Away Games Won"]:
                df.at[i, 'Home Points'] = row["Home Score"] + 1
                df.at[i, 'Away Points'] = row["Away Score"]
            elif row["Home Games Won"] < row["Away Games Won"]:
                df.at[i, 'Home Points'] = row["Home Score"]
                df.at[i, 'Away Points'] = row["Away Score"] + 1
            else:
                print(f"Error: No winner found for {row['Home Team']} vs {row['Away Team']}")

    return df

# Apply the function to the dataframe
results_df = create_points_columns(results_df)

In [179]:
monthly_results = results_df[results_df['Date'].dt.month == 10]

In [180]:
# Create a dataframe similar to hkcc_summary_df but based on the results in november_results_df
# To do this, we aggregate the Home Team Home Points won and Away Team Away Points won and combine them
# We also aggregate the number of games played by each team

# Create a dictionary to store the points won by each team
points_won = {}
games_played = {}
games_won = {}

# Iterate over the rows in november_results_df
for i, row in monthly_results.iterrows():
    home_team = row['Home Team']
    away_team = row['Away Team']
    home_points = row['Home Points']
    away_points = row['Away Points']

    # If the team is not in the dictionary, add it
    if home_team not in points_won:
        points_won[home_team] = 0
    if away_team not in points_won:
        points_won[away_team] = 0

    # Add the points won to the dictionary
    points_won[home_team] += home_points
    points_won[away_team] += away_points

    # Add games played to the dictionary
    if home_team not in games_played:
        games_played[home_team] = 0
    if away_team not in games_played:
        games_played[away_team] = 0

    games_played[home_team] += 1
    games_played[away_team] += 1

    # Add games won to the dictionary
    if home_team not in games_won:
        games_won[home_team] = 0
    if away_team not in games_won:
        games_won[away_team] = 0
    
    if row["Home Win"] == 1:
        games_won[home_team] += 1
    else:
        games_won[away_team] += 1

# Create a dataframe from the dictionary
monthly_summary_df = pd.DataFrame(list(points_won.items()), columns=['Team', 'Points'])
monthly_summary_df['Played'] = monthly_summary_df['Team'].map(games_played)
monthly_summary_df['Won'] = monthly_summary_df['Team'].map(games_won)
monthly_summary_df['Lost'] = monthly_summary_df['Played'] - monthly_summary_df['Won']

# Drop Team rows that don't contain "Hong Kong Cricket Club" or "HKCC"
monthly_summary_df = monthly_summary_df[(monthly_summary_df["Team"].str.contains(hkcc)) |
                                         (monthly_summary_df["Team"].str.contains("hkcc", case=False))]

# Create Division column by using Team column of november_summary_df and hkcc_summary_df
monthly_summary_df["Division"] = monthly_summary_df["Team"].map(hkcc_summary_df.set_index("Team")["Division"])

monthly_summary_df = monthly_summary_df[["Division", "Team", "Played", "Won", "Lost", "Points"]]

# Sort by Division based on the configured division order
division_order = list(all_divisions.keys())
monthly_summary_df["Division"] = pd.Categorical(
    monthly_summary_df["Division"],
    categories=division_order,
    ordered=True,
)

monthly_summary_df = monthly_summary_df.sort_values(by=["Division", "Team"], ascending=[True, True])

monthly_summary_df

,Division,Team,Played,Won,Lost,Points
9,2,Hong Kong Cricket Club D2,2,0,2,2
16,6,Hong Kong Cricket Club D6,3,1,2,7
1,10,Hong Kong Cricket Club D10,2,1,1,8
11,4,Hong Kong Cricket Club D4A,2,0,2,2
12,4,Hong Kong Cricket Club D4B,2,1,1,7
25,L2,Hong Kong Cricket Club L2,2,1,1,6
19,7,Hong Kong Cricket Club D7A,3,2,1,10
20,7,Hong Kong Cricket Club D7B,3,1,2,7
40,M2,Hong Kong Cricket Club M2,3,3,0,14
55,Premier Main,Hong Kong Cricket Club D1,1,1,0,3


In [181]:
# If 'Home Team' contains 'Hong Kong Cricket Club' or 'HKCC', then summarize the Home Points won for each Home Team. Do the same for the Away Team.
# Then combine the two dataframes into one.

hkcc_home_points = results_df[results_df['Home Team'].str.contains(hkcc) 
                              | results_df['Home Team'].str.contains('hkcc', case=False)].groupby('Home Team')['Home Points'].sum()

hkcc_away_points = results_df[results_df['Away Team'].str.contains(hkcc) 
                                | results_df['Away Team'].str.contains('hkcc', case=False)].groupby('Away Team')['Away Points'].sum()

hkcc_points_df = pd.concat([hkcc_home_points, hkcc_away_points], axis=1).fillna(0)

In [182]:
hkcc_points_df

,Home Points,Away Points
Hong Kong Cricket Club D10,6.0,2.0
Hong Kong Cricket Club D15,16.0,4.0
Hong Kong Cricket Club D4A,1.0,1.0
Hong Kong Cricket Club D4B,2.0,5.0
Hong Kong Cricket Club D7A,10.0,0.0
Hong Kong Cricket Club L2,5.0,1.0
Hong Kong Cricket Club L3,2.0,7.0
Hong Kong Cricket Club L4,4.0,8.0
Hong Kong Cricket Club M1A,5.0,4.0
Hong Kong Cricket Club M1B,1.0,2.0


In [183]:
results_df[(results_df["Home Score"] == results_df["Away Score"]) & 
           (results_df["Home Games Won"] == results_df["Away Games Won"])]

,Home Team,vs,Away Team,Venue,Time,Match Week,Date,Division,Overall Score,Rubbers,Home Score,Away Score,Home Games Won,Away Games Won,Home Win,Home Points,Away Points


In [184]:
round(hkcc_summary_df["Won"].sum() / hkcc_summary_df["Played"].sum(), 3)

0.566

### Show HKCC results that haven't been uploaded yet

In [185]:
# Filter rows where Home Team column or Away Team column contains "Hong Kong Cricket Club" or "hkcc" (case-insensitive)
hkcc_awaiting_results_df = awaiting_results_df[(awaiting_results_df["Home Team"].str.contains(hkcc, case=False)) | 
                                               (awaiting_results_df["Away Team"].str.contains(hkcc, case=False)) |
                                               (awaiting_results_df["Home Team"].str.contains("hkcc", case=False)) |
                                                (awaiting_results_df["Away Team"].str.contains("hkcc", case=False))]


In [186]:
hkcc_awaiting_results_df.sort_values("Date")

,Home Team,Away Team,Venue,Match Week,Date,Division
3,Hong Kong Cricket Club D4A,Sha Tin 1,Hong Kong Cricket Club,3,2025-10-28,4
4,X-Alpha 2,Hong Kong Cricket Club D4B,Cornwall Street Squash Centre,3,2025-10-28,4
7,Hong Kong Cricket Club L2,Hong Kong Football Club L2A,Hong Kong Cricket Club,3,2025-10-28,L2
11,Hong Kong Football Club M1B,Hong Kong Cricket Club M1A,Hong Kong Football Club,5,2025-10-30,Premier Masters
13,Hong Kong Cricket Club M4,SKCSC M4,Hong Kong Cricket Club,5,2025-10-30,M4
